# 作業七
## 題目一 (20%)
### 題目敘述

你正在處理一場技術會議的參與者資料。這些資料分布在兩個不同的系統中：
- 註冊系統：記錄了參與者的個人資料及其註冊狀況。
- 支付系統：記錄了所有支付交易。

你的任務是處理這些數據，為會議的財務團隊和註冊團隊提供他們需要的報告。

In [1]:
# data(dictionary)
registrations_data = {
    'participant_id': [101, 102, 103, 104, 105, 106, 107, 108, 109, 110],
    'name': ['Alice', 'Bob', 'Caroline', 'David', 'Eve', 'Frank', 'Grace', 'Heidi', 'Ivan', 'Judy'],
    'session_registered': ['Python', 'JavaScript', 'Java', 'Python', 'JavaScript', 'Java', 'Python', 'JavaScript', 'Java', 'Python']
}

transactions_data = {
    'transaction_id': [500, 501, 502, 503, 504, 505, 506, 507, 508, 509],
    'participant_id': [101, 105, 106, 107, 103, 108, 109, 110, 111, 112],  
    'amount': [100, 100, 100, 100, 100, 100, 100, 100, 100, 100],
    'status': ['Success', 'Failed', 'Success', 'Failed', 'Success', 'Failed', 'Success', 'Success', 'Failed', 'Failed']
}

In [2]:
import pandas as pd
registrations=pd.DataFrame(registrations_data)
transactions=pd.DataFrame(transactions_data)

In [3]:
df1=pd.merge(registrations, transactions, how="inner")
print("既有註冊紀錄又有繳費紀錄的參與者（不論繳費成功與否）")
print(df1)

既有註冊紀錄又有繳費紀錄的參與者（不論繳費成功與否）
   participant_id      name session_registered  transaction_id  amount  \
0             101     Alice             Python             500     100   
1             103  Caroline               Java             504     100   
2             105       Eve         JavaScript             501     100   
3             106     Frank               Java             502     100   
4             107     Grace             Python             503     100   
5             108     Heidi         JavaScript             505     100   
6             109      Ivan               Java             506     100   
7             110      Judy             Python             507     100   

    status  
0  Success  
1  Success  
2   Failed  
3  Success  
4   Failed  
5   Failed  
6  Success  
7  Success  


In [4]:
df2=pd.merge(registrations, transactions, how="left")
print("所有有註冊紀錄的參與者（包括那些沒有繳費紀錄的）")
print(df2)

所有有註冊紀錄的參與者（包括那些沒有繳費紀錄的）
   participant_id      name session_registered  transaction_id  amount  \
0             101     Alice             Python           500.0   100.0   
1             102       Bob         JavaScript             NaN     NaN   
2             103  Caroline               Java           504.0   100.0   
3             104     David             Python             NaN     NaN   
4             105       Eve         JavaScript           501.0   100.0   
5             106     Frank               Java           502.0   100.0   
6             107     Grace             Python           503.0   100.0   
7             108     Heidi         JavaScript           505.0   100.0   
8             109      Ivan               Java           506.0   100.0   
9             110      Judy             Python           507.0   100.0   

    status  
0  Success  
1      NaN  
2  Success  
3      NaN  
4   Failed  
5  Success  
6   Failed  
7   Failed  
8  Success  
9  Success  


In [5]:
df3=pd.merge(registrations, transactions, how="right")
print("所有有繳費紀錄的參與者（包括那些沒有註冊紀錄的）")
print(df3)

所有有繳費紀錄的參與者（包括那些沒有註冊紀錄的）
   participant_id      name session_registered  transaction_id  amount  \
0             101     Alice             Python             500     100   
1             105       Eve         JavaScript             501     100   
2             106     Frank               Java             502     100   
3             107     Grace             Python             503     100   
4             103  Caroline               Java             504     100   
5             108     Heidi         JavaScript             505     100   
6             109      Ivan               Java             506     100   
7             110      Judy             Python             507     100   
8             111       NaN                NaN             508     100   
9             112       NaN                NaN             509     100   

    status  
0  Success  
1   Failed  
2  Success  
3   Failed  
4  Success  
5   Failed  
6  Success  
7  Success  
8   Failed  
9   Failed  


In [6]:
df4=pd.merge(registrations, transactions, how="outer")
print("所有參與者的註冊紀錄和繳費紀錄")
print(df4)

所有參與者的註冊紀錄和繳費紀錄
    participant_id      name session_registered  transaction_id  amount  \
0              101     Alice             Python           500.0   100.0   
1              102       Bob         JavaScript             NaN     NaN   
2              103  Caroline               Java           504.0   100.0   
3              104     David             Python             NaN     NaN   
4              105       Eve         JavaScript           501.0   100.0   
5              106     Frank               Java           502.0   100.0   
6              107     Grace             Python           503.0   100.0   
7              108     Heidi         JavaScript           505.0   100.0   
8              109      Ivan               Java           506.0   100.0   
9              110      Judy             Python           507.0   100.0   
10             111       NaN                NaN           508.0   100.0   
11             112       NaN                NaN           509.0   100.0   

     sta

## 題目二 (40%)

### 題目敘述

這題會用到來自Dcard的資料集，這個資料集包含以下六個欄位：

- title: 標題
- commentCount: 留言數
- likeCount: 按讚數
- forumName: 看板名稱
- gender: 發文者性別
- text: 內文，格式是list of list of list (以字串資料型態呈現)
  - 第一層是整篇文章
  - 第二層是句子
  - 第三層是 `[token, POS tag]`

[Unicode 15.1 Character Code Charts](https://www.unicode.org/charts/)

In [7]:
# 1. 讀取檔案
import csv
dcard_df=pd.read_csv('./dcard-500.csv', encoding='utf-8')

In [8]:
# 2. 將gender欄位中的數值換成字母
dcard_df["gender"]=dcard_df["gender"].replace({0:'F', 1:'M'})

In [9]:
# 3. 新增欄位
# 抓token
i=0
dcard_df["reconstructed_text"]=""
for text in dcard_df["text"]:
    list_sen=eval(text)
    string_token=""
    for sentence in list_sen:
        if sentence != list_sen[0]: string_token+=' '
        for ele in sentence:
            if ele != sentence[0]: string_token+=' '
            string_token+=ele[0]
    dcard_df["reconstructed_text"].iloc[i]=string_token
    i+=1

/tmp/ipykernel_26180/1005442751.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dcard_df["reconstructed_text"].iloc[i]=string_token


In [10]:
import re
# 抓emoji
i=0
dcard_df["emojis"]=""
for sentence in dcard_df["reconstructed_text"]:
    emojis = re.findall(r'[\U00002600-\U000027BF\U0001f600-\U0001f64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F900-\U0001F9FF\u2600-\u26FF\u2700-\u27BF]', sentence)
    dcard_df["emojis"].iloc[i]=''.join(emojis)
    i+=1
dcard_df.head()
# r'[\U00002600-\U000027BF\U0001f600-\U0001f64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F900-\U0001F9FF\u2600-\u26FF\u2700-\u27BF]+'

/tmp/ipykernel_26180/3770199301.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dcard_df["emojis"].iloc[i]=''.join(emojis)


,title,commentCount,likeCount,forumName,gender,text,date,reconstructed_text,emojis
0,想詢問一下,4,0,3C,F,"[[['Iphone', 'FW'], ['手機', 'Na'], ['line', 'FW...",2020-01-23,Iphone 手機 line 視訊 有 辦法 一邊 螢幕 錄製 一邊 視訊 嗎 因為 家人 ...,
1,#新市 #羊肉爐 阿旦師羊肉爐,5,54,美食,M,"[[['各位', 'Nh'], ['晚安', 'VH'], ['我', 'Nh'], ['是...",2020-01-13,各位 晚安 我 是 阿杰 ~ https://i.imgur.com/OU0rqqz . j...,
2,四面彿 靈驗,2,8,靈異,M,"[[['這陣子', 'Nd'], ['跟', 'P'], ['女友', 'Na'], ['分...",2020-01-22,這陣子 跟 女友 分分合合 的 之前 就 求 過 長春路 的 四面 彿 不少 事情 幾乎...,
3,Foodpanda到底是？,10,19,美食,F,"[[['訂', 'VC'], ['了', 'Di'], ['兩', 'Neu'], ['天'...",2020-01-29,訂 了 兩 天 ： 第一 天 ， 原本 寫 要 等 50 分鐘 ， 結果 後來 又 一直 改...,
4,#協尋匿名盜圖仔！ 當創作者塑膠？,9,50,手作,F,"[[['（', 'PARENTHESISCATEGORY'], ['❎', 'FW'], [...",2020-01-15,（ ❎ 協 尋 盜 圖 者 ❎ ） 我 需要 大家 的 幫忙 ！ 協尋 ： 新南威爾斯 大學...,❎❎📢📢


In [11]:
# 4. 計算emoji的使用次數
all_emojis = ''.join(dcard_df['emojis']).split()
emoji_series = pd.Series(list(all_emojis[0]))
emoji_count = emoji_series.value_counts()
top_10_emojis = emoji_count.head(10)

print(top_10_emojis)

😂    186
😭    109
🏻    103
❤     72
👇     66
🙏     62
🔴     45
🤣     38
♀     32
👧     31
dtype: int64


In [12]:
# 5. 根據不同性別分析前十個最常用的表情符號
# 取前十名的emoji列表，找出含有他們的資料
top_10=top_10_emojis.index.tolist()
top_10_df=top_10_emojis.to_frame().reset_index().rename(columns={'index': 'emoji'})

In [13]:
# Male
male=dcard_df[dcard_df["gender"]=='M']
male_emojis = ''.join(male['emojis']).split()
male_emoji_series = pd.Series(list(male_emojis[0]))
male_emoji_count = male_emoji_series.value_counts()

male_10_df=male_emoji_count.to_frame().reset_index().rename(columns={'index': 'emoji'})

In [14]:
# Female
female=dcard_df[dcard_df["gender"]=='F']
female_emojis = ''.join(female['emojis']).split()
female_emoji_series = pd.Series(list(female_emojis[0]))
female_emoji_count = female_emoji_series.value_counts()

female_10_df=female_emoji_count.to_frame().reset_index().rename(columns={'index': 'emoji'})

In [15]:
male_mg=pd.merge(top_10_df, male_10_df, on=["emoji"], how="left")
male_mg=male_mg.drop(columns=['0_x'])
male_mg=male_mg.rename(columns={'0_y':'M'})
male_mg

,emoji,M
0,😂,55.0
1,😭,12.0
2,🏻,14.0
3,❤,4.0
4,👇,17.0
5,🙏,13.0
6,🔴,NaN
7,🤣,5.0
8,♀,NaN
9,👧,NaN


In [16]:
female_mg=pd.merge(top_10_df, female_10_df, on=["emoji"], how="left")
female_mg=female_mg.drop(columns=['0_x'])
female_mg=female_mg.rename(columns={'0_y':'F'})
female_mg

,emoji,F
0,😂,131
1,😭,97
2,🏻,89
3,❤,68
4,👇,49
5,🙏,49
6,🔴,45
7,🤣,33
8,♀,32
9,👧,31


In [17]:
B=pd.merge(male_mg, female_mg, how="outer")
B

,emoji,M,F
0,😂,55.0,131
1,😭,12.0,97
2,🏻,14.0,89
3,❤,4.0,68
4,👇,17.0,49
5,🙏,13.0,49
6,🔴,NaN,45
7,🤣,5.0,33
8,♀,NaN,32
9,👧,NaN,31


In [18]:
# 縱向比較不同表情符號在不同性別中的使用次數
B.fillna(0, inplace=True)
B['M'] = B['M'].astype(int)
B.set_index("emoji", inplace=True)
display(B)

,M,F
emoji,,
😂,55,131
😭,12,97
🏻,14,89
❤,4,68
👇,17,49
🙏,13,49
🔴,0,45
🤣,5,33
♀,0,32


In [19]:
# 橫向比較不同性別對同一表情符號的使用次數
A=B.T
display(A)

emoji,😂,😭,🏻,❤,👇,🙏,🔴,🤣,♀,👧
M,55,12,14,4,17,13,0,5,0,0
F,131,97,89,68,49,49,45,33,32,31


## 題目三 (40%)

### 題目敘述

這題會用到MovieLens網站提供的資料集來進行電影推薦（[資料來源](https://grouplens.org/datasets/movielens/)）。這份資料包含了使用者的評分數據和電影的詳細資料。

In [20]:
# 1. 讀取資料
movies=pd.read_csv('./movies.csv', encoding='utf-8')
ratings=pd.read_csv('./ratings.csv', encoding='utf-8')

In [21]:
# 2. 前處理
# left join合併ratings和movies
ratings_movies=pd.merge(ratings, movies, on='movieId', how='left')
ratings_movies.head(5)


,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [22]:
# 選出userId為2的使用者
user2=ratings_movies[ratings_movies['userId']==2]
user2=user2.drop_duplicates(subset='movieId', keep='last')
user2.head(10)

,userId,movieId,rating,timestamp,title,genres
232,2,318,3.0,1445714835,"Shawshank Redemption, The (1994)",Crime|Drama
233,2,333,4.0,1445715029,Tommy Boy (1995),Comedy
234,2,1704,4.5,1445715228,Good Will Hunting (1997),Drama|Romance
235,2,3578,4.0,1445714885,Gladiator (2000),Action|Adventure|Drama
236,2,6874,4.0,1445714952,Kill Bill: Vol. 1 (2003),Action|Crime|Thriller
237,2,8798,3.5,1445714960,Collateral (2004),Action|Crime|Drama|Thriller
238,2,46970,4.0,1445715013,Talladega Nights: The Ballad of Ricky Bobby (2...,Action|Comedy
239,2,48516,4.0,1445715064,"Departed, The (2006)",Crime|Drama|Thriller
240,2,58559,4.5,1445715141,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX
241,2,60756,5.0,1445714980,Step Brothers (2008),Comedy


In [23]:
# 3. 推薦電影
genres_split = user2['genres'].str.split('|', expand=True).stack().reset_index(level=1, drop=True)
genres_split.name = 'genre'  
user2_genres = user2.drop('genres', axis=1).join(genres_split)

genre_rating_counts = user2_genres.groupby('genre')['rating'].count().reset_index()
genre_rating_counts.rename(columns={'rating': 'rating_counts'}, inplace=True)

filtered_genres = genre_rating_counts[genre_rating_counts['rating_counts'] >= 10]
user2_genres_filtered = user2_genres[user2_genres['genre'].isin(filtered_genres['genre'])]

user2_ratings = user2_genres_filtered.groupby('genre')['rating'].mean().reset_index()
user2_ratings = user2_ratings.sort_values(by='rating', ascending=False)

print(user2_ratings)


      genre    rating
0    Action  3.954545
2     Drama  3.882353
1     Crime  3.800000
3  Thriller  3.700000


In [24]:
top_genre = user2_ratings.iloc[0]['genre']

user2_seen_movies = set(user2['movieId'])  # user2已經看過的電影
recommended_movies = ratings_movies[(ratings_movies['genres'].str.contains(top_genre)) & (~ratings_movies['movieId'].isin(user2_seen_movies))]

recommended_movies = recommended_movies.drop_duplicates(subset='movieId').head(10)

print(recommended_movies[['movieId', 'title', 'genres']])


    movieId                                      title  \
2         6                                Heat (1995)   
5        70                 From Dusk Till Dawn (1996)   
7       110                          Braveheart (1995)   
8       151                             Rob Roy (1995)   
10      163                           Desperado (1995)   
15      260  Star Wars: Episode IV - A New Hope (1977)   
17      316                            Stargate (1994)   
19      349            Clear and Present Danger (1994)   
22      367                           Mask, The (1994)   
23      423                          Blown Away (1994)   

                           genres  
2           Action|Crime|Thriller  
5   Action|Comedy|Horror|Thriller  
7                Action|Drama|War  
8        Action|Drama|Romance|War  
10         Action|Romance|Western  
15        Action|Adventure|Sci-Fi  
17        Action|Adventure|Sci-Fi  
19    Action|Crime|Drama|Thriller  
22    Action|Comedy|Crime|Fantasy  
2